In [1]:
import pandas as pd

In [2]:
df_products = pd.read_csv(filepath_or_buffer='data/products.csv')
df_products.head()

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


In [3]:
# CategoryID göre ürünlerin stok durumu
# aggregate functions --> sum(), mean(), min(), max(), count()
(df_products.groupby(by='CategoryID')[['UnitsInStock']]
           .sum()
           .sort_values(by='UnitsInStock', ascending=False))

,UnitsInStock
CategoryID,
8,701
1,559
2,507
4,393
3,386
5,308
6,165
7,100


In [4]:
# Hangi tedarikçiden kaç tane ürün alıyorum
(df_products
    .groupby(by='SupplierID')[['ProductID']]
    .count()
    .sort_values(by='ProductID', ascending=False)
    .rename(
        columns={
            'ProductID': 'TotalProduct'
        }))

,TotalProduct
SupplierID,
7,5
12,5
8,4
2,4
1,3
11,3
4,3
3,3
6,3


In [5]:
df_orders = pd.read_csv(filepath_or_buffer='data/orders.csv')
df_orders.head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,04/07/1996,01/08/1996,16/07/1996,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NaN,51100,France
1,10249,TOMSP,6,05/07/1996,16/08/1996,10/07/1996,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,NaN,44087,Germany
2,10250,HANAR,4,08/07/1996,05/08/1996,12/07/1996,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,08/07/1996,05/08/1996,15/07/1996,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,NaN,69004,France
4,10252,SUPRD,4,09/07/1996,06/08/1996,11/07/1996,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,NaN,B-6000,Belgium


In [6]:
# hangi kargo firmasına ne kadar para ödemişim
(df_orders
    .groupby(by='ShipVia')[['Freight']]
    .sum()
    .sort_values(by='Freight', ascending=False))

,Freight
ShipVia,
2,28244.85
3,20512.51
1,16185.33


In [7]:
# hangi çalışan kaç tane sipariş gerçekleştirmiş.
(df_orders
    .groupby(by='EmployeeID')[['OrderID']]
    .count()
    .sort_values(by='OrderID', ascending=False)
    .rename(
        columns={
            'OrderID': 'TotalProduct'
        }))    

,TotalProduct
EmployeeID,
4,156
3,127
1,123
8,104
2,96
7,72
6,67
9,43
5,42


In [8]:
# hangi müşteri kaç tane sipariş yapmış
df_grouped = (df_orders
                .groupby(by='CustomerID')[['OrderID']]
                .count()
                .sort_values(by='OrderID', ascending=False)
                .rename(
                     columns={
                        'OrderID': 'TotalOrder'
                    }
                ))
# df_grouped['CustomerID'] --> kodu ile ilgili veri setinde "CustomerID" adında yeni bir sütun açtık.
# df_grouped.index --> kodu "df_grouped" veri setinin hali hazırda var olan index bilgilerini aldık. aynı "df_grouped.columns" da sütun isimlerini almamız gibi.
df_grouped['CustomerID'] = df_grouped.index
# reset_index() --> fonksiyonu ile var olan indexleri sıfırlayarak 0'dan başlayıp pozitif yönde artan index numarası verilir.
df_grouped.reset_index(drop=True)
 

,TotalOrder,CustomerID
0,31,SAVEA
1,30,ERNSH
2,28,QUICK
3,19,FOLKO
4,19,HUNGO
...,...,...
84,3,NORTS
85,3,THECR
86,2,LAZYK
87,2,GROSR


In [9]:
# ürünleri categorilerine göre gruplayalım ve stok miktarı 50 altında olanları listeleyelim
# Path I
(df_products
    .groupby(by='CategoryID')
    .agg({'UnitsInStock': 'sum'})
    .sort_values(by='UnitsInStock', ascending=False)
    .query('UnitsInStock < 500'))
    

,UnitsInStock
CategoryID,
4,393
3,386
5,308
6,165
7,100


In [ ]:
# Path II
(df_products.groupby(by='CategoryID')[['UnitsInStock']]
            .sum() # elemanlarının toplamını döndürür
            .sort_values(by='UnitsInStock', ascending=False)
            .query('UnitsInStock < 500'))


,UnitsInStock
CategoryID,
4,393
3,386
5,308
6,165
7,100


In [ ]:
# tedarikçilere göre ürünlerin stok ortalaması nedir ve ortalaması 50'den küçük olanlar filtlerensin
(df_products
    .groupby(by='SupplierID')
    .agg({'UnitsInStock': 'mean'})
    .sort_values(by='UnitsInStock', ascending=False)
    .query('UnitsInStock < 50'))

,UnitsInStock
SupplierID,
28,49.000000
3,47.000000
11,46.666667
23,44.000000
18,43.000000
12,41.000000
2,33.250000
6,32.666667
26,28.500000


In [12]:
df_od = pd.read_csv(filepath_or_buffer='data/order_details.csv')
df_od.head()

,OrderID,ProductID,UnitPrice,Quantity,Discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0
4,10249,51,42.4,40,0.0


In [16]:
df_od.shape

(2155, 6)

In [13]:
# Features Creating
# Projedeki ihtiyaçlara göre var olan sütunlardaki veriler üzerinde aritmatiksel işlemler ya da string manupülasyonları yapılarak yeni özellik yaratılma işlemine denir.
# df_od sütununda olmayan "Revenue" sütunu craft edelim. Bunun için UnitPrice * Quantity * (1- Discount) formülünü kullanalım. 
df_od['Revenue'] = df_od['UnitPrice'] * df_od['Quantity'] * (1- df_od['Discount'])
df_od.head()

,OrderID,ProductID,UnitPrice,Quantity,Discount,Revenue
0,10248,11,14.0,12,0.0,168.0
1,10248,42,9.8,10,0.0,98.0
2,10248,72,34.8,5,0.0,174.0
3,10249,14,18.6,9,0.0,167.4
4,10249,51,42.4,40,0.0,1696.0


In [18]:
# Siparişlere göre toplam cirom nedir, 2000 den küçük 1000 den büyük olanları sıralayalım
(df_od
    .groupby(by='OrderID')
    .agg({'Revenue': 'sum'})
    .sort_values(by='Revenue', ascending=False)
    .query('Revenue < 2000 and Revenue > 1000'))

,Revenue
OrderID,
10436,1994.520
10593,1994.400
10756,1990.000
10871,1979.230
10830,1974.000
...,...
11026,1030.000
10944,1025.325
10423,1020.000


In [ ]:
# En yüksek ortalama gelir getiren ürünleri bulup listeleyelim
(df_od
    .groupby('ProductID')
    .agg(AvgRevenue=('Revenue', 'mean'))
    .sort_values(by='AvgRevenue', ascending=False)
    .head(5)
)


,AvgRevenue
ProductID,
38,5891.530625
29,2511.521000
27,1677.763889
9,1445.300000
20,1410.210000


In [ ]:
# Her siparişte toplam kaç ürün satılmış listeleyelim
(df_od
    .groupby('OrderID')
    .agg(TotalQuantity=('Quantity', 'sum'))
    .sort_values(by='TotalQuantity', ascending=False)
    .head()
)


,TotalQuantity
OrderID,
10895,346
11030,330
10847,288
10515,286
10678,280


In [ ]:
# İndirim uygulanan siparişlerden toplam gelir nedir listeleyelim
(df_od[df_od['Discount'] > 0]
    .groupby('OrderID')
    .agg(TotalRevenue=('Revenue', 'sum'))
    .sort_values(by='TotalRevenue', ascending=False)
)



,TotalRevenue
OrderID,
10865,16387.50
11030,11120.55
10372,9210.90
10424,9194.56
10353,8593.28
...,...
10960,33.75
10586,23.80
10835,20.80


In [ ]:
# Birim fiyatı 20’den büyük ve indirimli satılmış ürünler nelerdir listeleyelim
df_od.query('UnitPrice > 20 and Discount > 0')


,OrderID,ProductID,UnitPrice,Quantity,Discount,Revenue
6,10250,51,42.40,35,0.15,1261.4000
11,10252,20,64.80,40,0.05,2462.4000
31,10258,32,25.60,6,0.20,122.8800
36,10260,62,39.40,15,0.25,443.2500
45,10263,30,20.70,60,0.25,931.5000
...,...,...,...,...,...,...
2137,11077,12,38.00,2,0.05,72.2000
2139,11077,14,23.25,1,0.03,22.5525
2141,11077,20,81.00,1,0.04,77.7600
2149,11077,60,34.00,2,0.06,63.9200


In [ ]:
# Siparişlerin ortalama birim fiyatı 30’dan büyük olanlarını listeleyelim
(df_od
    .groupby('OrderID')
    .agg(AvgUnitPrice=('UnitPrice', 'mean'))
    .query('AvgUnitPrice > 30')
    .sort_values(by='AvgUnitPrice', ascending=False)
)


,AvgUnitPrice
OrderID,
10981,263.500000
10828,172.250000
10816,156.400000
10672,142.500000
10889,142.250000
...,...
10266,30.400000
10433,30.400000
10295,30.400000


In [ ]:
# Yeni bir özellik (RevenuePerItem) oluştur ve en yüksek 5 satırı gösterelim
df_od['RevenuePerItem'] = df_od['Revenue'] / df_od['Quantity']
df_od.sort_values(by='RevenuePerItem', ascending=False).head()


,OrderID,ProductID,UnitPrice,Quantity,Discount,Revenue,RevenuePerItem
2027,11032,38,263.5,25,0.0,6587.5,263.5
1678,10889,38,263.5,40,0.0,10540.0,263.5
1459,10805,38,263.5,10,0.0,2635.0,263.5
1403,10783,38,263.5,5,0.0,1317.5,263.5
1511,10828,38,263.5,2,0.0,527.0,263.5
